<a href="https://colab.research.google.com/github/ShinWalnut/ABR_basic/blob/master/ABR_HW4_MNIST_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

#데이터 불러오기
mnist = input_data.read_data_sets("MNIST/", one_hot=True)

# parameter
epoch = 10
mini_batch_size = 500
total_batch_size = int(mnist.train.num_examples / mini_batch_size)

Extracting MNIST/train-images-idx3-ubyte.gz
Extracting MNIST/train-labels-idx1-ubyte.gz
Extracting MNIST/t10k-images-idx3-ubyte.gz
Extracting MNIST/t10k-labels-idx1-ubyte.gz


In [0]:
### 모델 생성

## place holder 생성
X = tf.placeholder(shape = [None, 28, 28, 1], dtype = tf.float32) # MNIST 이미지[28,28,1] 데이터 입력
Y = tf.placeholder(shape = [None, 10], dtype = tf.float32) # MNIST 라벨 데이터 입력

## 모델 레이어 구조 생성
# output 이미지의 사이즈 계산 : NxN image * FxF filter * SxS strides => (N-F)/S + 1 (계산 값이 정수가 되도록 설정)
# conv(@20) - max_pool- conv(@32) - average_pool - mlp(분류)
conv_1 = tf.layers.conv2d(X, filters = 20, kernel_size = [3, 3], strides = [1, 1], padding = "same", activation = tf.nn.relu) #batch x 28 x 28 @ 20
pool_1 = tf.layers.max_pooling2d(conv_1, pool_size = [2, 2], strides = [2, 2], padding = "valid") #batch x 14 x 14 @ 20

conv_2 = tf.layers.conv2d(pool_1, filters = 32, kernel_size = [3, 3], strides = [1, 1], padding = "valid", activation = tf.nn.relu) #batch x 12 x 12 @ 32
pool_2 = tf.layers.average_pooling2d(conv_2, pool_size = [2, 2], strides = [2, 2], padding= "valid") #batch x 6 x 6 @ 32

## 분류를 위한 MLP 출력단
flat = tf.layers.flatten(inputs = pool_2)
h_layer1 = tf.layers.dense(flat, 512, activation = tf.nn.relu)
h_layer2 = tf.layers.dense(h_layer1, 128, activation = tf.nn.relu)
output_layer = tf.layers.dense(h_layer2, 10)

## loss training model
loss = tf.losses.softmax_cross_entropy(onehot_labels = Y, logits = output_layer)
train_optimizer = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)

## test를 위한 정확도 측정
predict = tf.nn.softmax(output_layer) # 모델을 통해 출력된 데이터를 softmax에 통과시켜 모델이 예측한 값을 찾는다 
correct = tf.cast(tf.equal(tf.argmax(Y, 1), tf.argmax(predict, 1)), tf.float32) # 라벨과 예측값이 맞다면 equl()은 True를 반환, 이를 cast()가 1로 바꾸어준다. (false는 0으로 바꾼다)
accuracy = tf.reduce_mean(correct) # correct의 평균을 구한다. 0 ~ 1사이의 값. 

In [29]:
#세션 시작
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  for epo in range(epoch):
    avg_train_loss, avg_valid_loss = 0, 0
    
    for bat in range(total_batch_size):
      #batch 할당
      train_xdata, train_labels = mnist.train.next_batch(mini_batch_size)
      train_xdata = np.reshape(train_xdata, [-1, 28, 28, 1])
      valid_xdata, valid_labels = mnist.validation.next_batch(mini_batch_size)
      valid_xdata = np.reshape(valid_xdata, [-1, 28, 28, 1])
      
      #세션 실행
      train_loss, _ = sess.run([loss, train_optimizer], feed_dict={X: train_xdata, Y: train_labels})
      valid_loss = sess.run(loss, feed_dict= {X: valid_xdata, Y: valid_labels})
      avg_train_loss += train_loss / total_batch_size
      avg_valid_loss += valid_loss / total_batch_size
      
      acc_train = sess.run(accuracy, feed_dict={X: train_xdata, Y: train_labels})
      acc_valid = sess.run(accuracy, feed_dict={X: valid_xdata, Y: valid_labels})
      if epo==0 or (bat+1)%(total_batch_size/2)==0:
        print("Epoch :", '%02d' % (epo+1), "Batch :",'%03d'%(bat+1), ", avg_train_loss = ", "{:.9f}".format(avg_train_loss), "(", "{:.3f}%".format(100*acc_train), ")",
          ", avg_valid_loss = ", "{:.9f}".format(avg_valid_loss), "(", "{:.3f}%".format(100*acc_valid), ")")
  
  print("학습 종료")
  
  test_xdata, test_labels = mnist.test.next_batch(mini_batch_size)
  test_xdata = np.reshape(test_xdata, [-1, 28, 28, 1])
  print("Accuracy : {:.3f}%".format(100.*sess.run(accuracy, feed_dict={X: test_xdata, Y: test_labels})))

Epoch : 01 Batch : 001 , avg_train_loss =  0.020986585 ( 40.800% ) , avg_valid_loss =  0.020420445 ( 36.200% )
Epoch : 01 Batch : 002 , avg_train_loss =  0.041403148 ( 58.400% ) , avg_valid_loss =  0.040231499 ( 52.800% )
Epoch : 01 Batch : 003 , avg_train_loss =  0.061251207 ( 57.400% ) , avg_valid_loss =  0.059411814 ( 52.000% )
Epoch : 01 Batch : 004 , avg_train_loss =  0.080420438 ( 54.600% ) , avg_valid_loss =  0.077744976 ( 55.600% )
Epoch : 01 Batch : 005 , avg_train_loss =  0.098858448 ( 59.800% ) , avg_valid_loss =  0.095124383 ( 63.800% )
Epoch : 01 Batch : 006 , avg_train_loss =  0.116519209 ( 66.600% ) , avg_valid_loss =  0.111427012 ( 67.600% )
Epoch : 01 Batch : 007 , avg_train_loss =  0.132791561 ( 72.800% ) , avg_valid_loss =  0.126587278 ( 65.800% )
Epoch : 01 Batch : 008 , avg_train_loss =  0.148048479 ( 71.800% ) , avg_valid_loss =  0.140607062 ( 69.000% )
Epoch : 01 Batch : 009 , avg_train_loss =  0.161887730 ( 71.000% ) , avg_valid_loss =  0.153150717 ( 68.200% )
E